![scientisst-notebooks_top-banner](https://raw.githubusercontent.com/scientisst/notebooks/59632d3d477981a3b1cc12157e12bbdcdb45def8/_Resources/top-banner.png)

# <span style="color:#484848;"> LAB 1: ECG Preprocessing and HRV Feature Extraction </span>

### <span style='color:#00aba1;'> Keywords </span>

`ECG`, `BioSSPy`, `Feature Extraction`, `HRV`

Name:

IST Number:

### <span style='color:#00aba1;'> Notebook Info </span>

**Contributor(s):** Rafael Silva, Hugo Plácido da Silva, Ana Fred, and Vicente Garção

**Date of creation:** 13/06/2022

**Last update:** 26/08/2023

### <span style="color:#00aba1;"> Description & Objectives </span>
The purpose of this lab is to understand how to extract meaningful information from data in the form of features:

* Understand the concept of features in the context of biosignals
* Understand and visualize the ECG signal
* Understand the feature extraction process for Heart Rate Variability (HRV)
* Analyse and compare HRV features for different conditions (healthy subjects vs atrial fibrillation)

### <span style="color:#00aba1;"> Requirements </span>
In this section, the libraries required should be installed, using the command:

In [ ]:
!pip install biosppy
!pip install astropy
!pip install seaborn

and imported:

In [ ]:
# NumPy is the fundamental package for scientific computing with Python. 
import numpy as np 
# Matplotlib is a Python 2D plotting library
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# BioSPPy is a toolbox for biosignal processing
import biosppy as bp

# <span style="color:#00aba1;"> 1. Introduction </span>

## <div style="color:#00aba1;">  1.1 Feature Extraction for ECG </div>

One of the most important concepts in Machine Learning is that of 'features', which are individual measurable properties or characteristics of the data, which we can use as variables in ML models. Often, the success of a model hinges on the quality and relevance of the available features. Extracting the right features from raw data can allow models to discern patterns, recognize relationships, make predictions, or classify data.

Biosignals, like ECG, are rich sources of data, but their raw form can be overwhelming for ML models. By transforming these signals into a set of meaningful yet simple bits of information, we can capture the essence of the data while shedding the noise and redundancy. 

Heart Rate Variability (HRV) analysis is the most prominent method of feature extraction from ECG signals. HRV examines the variations in time between successive heartbeats, offering insights into autonomic nervous system activity and potential cardiac anomalies.

In this class, we'll explore the art and science of feature extraction, with a special focus on HRV analysis. By understanding these foundational concepts, you'll be better equipped to harness the power of ML for biomedical research and applications.

## <div style="color:#00aba1;">  1.2 The ECG Signal </div>

The Electrocardiogram (ECG/EKG) is a graphical representation of the electrical activity of the heart over time. It's a vital tool in cardiology, providing insights into heart rhythm, blood flow, and muscle function. The ECG signal is typically acquired using electrodes placed on the skin, which detect the electrical impulses generated by the polarization and depolarization of cardiac tissue.

Key Characteristics/Points of an ECG Signal:

P Wave: Represents the electrical impulse that initiates the heartbeat, corresponding to the depolarization of the atria (upper chambers of the heart).

QRS Complex:

Q Wave: The first downward deflection after the P wave.
R Wave: The upward deflection, which is the peak of the QRS complex.
S Wave: The downward deflection following the R wave.
The QRS complex signifies the depolarization of the ventricles (lower chambers of the heart) and is the most prominent feature of the ECG trace.

T Wave: Represents the repolarization (or recovery) of the ventricles. It's an upward deflection following the QRS complex.

PR Interval: The time interval from the onset of the P wave to the start of the QRS complex, indicating the time the electrical impulse takes to travel from the sinus node through the atrioventricular node.

QT Interval: The time from the start of the Q wave to the end of the T wave, representing the time taken for the ventricles to depolarize and then repolarize.



![ECG_points](https://raw.githubusercontent.com/scientisst/notebooks/master/F.Machine%20Learning%20and%20Knowledge%20Extraction/F005%20Biosignal%20Feature%20Extraction/_Resources/gram-1024x723.png)


# <span style="color:#00aba1;"> 2. ECG Processing & Analysis </span>

## <span style="color:#00aba1;"> 2.1. Visualizing the Raw ECG Data </span>

In this lab we will analyse and compare ECG data from a healthy subject and from a subject with Atrial Fibrillation (Afib), a type of arrhythmia. We will use the BioSPPY package for Python, specifically its ECG module. BioSPPY provides a wide variety of tools for biosignal processing, 

With this module we can visualize, filter, and segment the ECG signal, as well as compute some important information about the signal which will let us extract meaningful features.

Let's load an ECG file for a healthy subject and use the function `signals.ecg.ecg()`, that automatically filters the signal, finds the R-peaks, computes the instantaneous heart rate and segments the ECG waveforms. This function also provides an interactive interface to visualize its outputs.

In [ ]:
# Load the ECG data
fs = 1000. # Hz
ecg_op3 = np.loadtxt('https://raw.githubusercontent.com/scientisst/notebooks/93ed88e922dc831a34f766675355eedd9f220244/F.Machine%20Learning%20and%20Knowledge%20Extraction/F005%20Biosignal%20Feature%20Extraction/_Resources/ecg_op3.csv')
ecg_af = np.loadtxt('https://raw.githubusercontent.com/scientisst/notebooks/93ed88e922dc831a34f766675355eedd9f220244/F.Machine%20Learning%20and%20Knowledge%20Extraction/F005%20Biosignal%20Feature%20Extraction/_Resources/ecg_af.csv')

# BioSPPy
bp.signals.ecg.ecg(ecg_op3, fs, show=True, interactive=False);

This provides us with a lot of information, which may be tough to interpret at first glance. 

As mentioned, these plots were obtained using the `signals.ecg.ecg()` function:

```python
def ecg(signal=None, sampling_rate=1000.0, path=None, show=True, interactive=False):
    """Process a raw ECG signal and extract relevant signal features using
    default parameters.

    Parameters
    ----------
    signal : array
        Raw ECG signal.
    sampling_rate : int, float, optional
        Sampling frequency (Hz).
    path : str, optional
        If provided, the plot will be saved to the specified file.
    show : bool, optional
        If True, show a summary plot.
    interactive : bool, optional
        If True, shows an interactive plot.

    Returns
    -------
    ts : array
        Signal time axis reference (seconds).
    filtered : array
        Filtered ECG signal.
    rpeaks : array
        R-peak location indices.
    templates_ts : array
        Templates time axis reference (seconds).
    templates : array
        Extracted heartbeat templates.
    heart_rate_ts : array
        Heart rate time axis reference (seconds).
    heart_rate : array
        Instantaneous heart rate (bpm).

    """

    # check inputs
    if signal is None:
        raise TypeError("Please specify an input signal.")

    # ensure numpy
    signal = np.array(signal)

    sampling_rate = float(sampling_rate)

    # filter signal
    order = int(1.5 * sampling_rate)
    filtered, _, _ = st.filter_signal(
        signal=signal,
        ftype="FIR",
        band="bandpass",
        order=order,
        frequency=[0.67, 45],
        sampling_rate=sampling_rate,
    )

    filtered = filtered - np.mean(filtered)  # remove DC offset

    # segment
    (rpeaks,) = hamilton_segmenter(signal=filtered, sampling_rate=sampling_rate)

    # correct R-peak locations
    (rpeaks,) = correct_rpeaks(
        signal=filtered, rpeaks=rpeaks, sampling_rate=sampling_rate, tol=0.05
    )

    # extract templates
    templates, rpeaks = extract_heartbeats(
        signal=filtered,
        rpeaks=rpeaks,
        sampling_rate=sampling_rate,
        before=0.2,
        after=0.4,
    )

    # compute heart rate
    hr_idx, hr = st.get_heart_rate(
        beats=rpeaks, sampling_rate=sampling_rate, smooth=True, size=3
    )

    # get time vectors
    length = len(signal)
    T = (length - 1) / sampling_rate
    ts = np.linspace(0, T, length, endpoint=True)
    ts_hr = ts[hr_idx]
    ts_tmpl = np.linspace(-0.2, 0.4, templates.shape[1], endpoint=False)

    # plot
    if show:
        if interactive:
            inter_plotting.plot_ecg(
                ts=ts,
                raw=signal,
                filtered=filtered,
                rpeaks=rpeaks,
                templates_ts=ts_tmpl,
                templates=templates,
                heart_rate_ts=ts_hr,
                heart_rate=hr,
                path=path,
                show=True,
            )

        else:
            plotting.plot_ecg(
                ts=ts,
                raw=signal,
                filtered=filtered,
                rpeaks=rpeaks,
                templates_ts=ts_tmpl,
                templates=templates,
                heart_rate_ts=ts_hr,
                heart_rate=hr,
                path=path,
                show=True,
            )

    # output
    args = (ts, filtered, rpeaks, ts_tmpl, templates, ts_hr, hr)
    names = (
        "ts",
        "filtered",
        "rpeaks",
        "templates_ts",
        "templates",
        "heart_rate_ts",
        "heart_rate",
    )

    return utils.ReturnTuple(args, names)

```

In order to understand every step of the processing pipeline for the ECG signals as computed by the `signals.ecg.ecg()` function, as well as identify the biggest differences between healthy and afib ECG data, we will visualize and analyze each step individually.

First, let's plot the raw signals.

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(15, 12))

# Plotting the raw ECG data for normal condition
axes[0].plot(ecg_op3[:8000], label='Normal ECG', color='blue')
axes[0].set_title('Raw ECG Data - Normal Condition')
axes[0].set_xlabel('Time (ms)')
axes[0].set_ylabel('Amplitude')
axes[0].legend()
axes[0].grid(True)

# Plotting the raw ECG data for atrial fibrillation (Afib) condition
axes[1].plot(ecg_af[:8000], label='AF ECG', color='red')
axes[1].set_title('Raw ECG Data - Atrial Fibrillation (Afib) Condition')
axes[1].set_xlabel('Time (ms)')
axes[1].set_ylabel('Amplitude')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

## <span style="color:#00aba1;"> 2.2. Filtering & Detecting R-Peaks </span>

The first steps in the ECG processing pipeline consist of filtering and detecting the R-peaks.

Raw ECG signals often contain noise from various sources, such as muscle activity, power line interference, and baseline wander due to respiration. To obtain a clear representation of the heart's electrical activity, it's important to filter out these unwanted components.

In this step, a bandpass filter is applied to the ECG signal to retain only the frequencies of interest and remove the noise.

The R-Peaks are the most prominent points in the ECG waveform and correspond to the ventricular contraction. Detecting R-peaks is crucial for segmenting the ECG signal into individual heartbeats and for further analysis like HRV computation.

In [ ]:
# Using the biosppy's ecg function to filter the ECG signal
out_normal = bp.signals.ecg.ecg(ecg_op3, sampling_rate=fs, show=False)
out_af = bp.signals.ecg.ecg(ecg_af, sampling_rate=fs, show=False)

fig, axes = plt.subplots(2, 1, figsize=(15, 12))

# Plotting the filtered ECG data with detected R-peaks for normal condition
axes[0].plot(out_normal['ts'][:8000], out_normal['filtered'][:8000], label='Filtered Normal ECG', color='blue')
axes[0].plot(out_normal['ts'][out_normal['rpeaks'][:10]], out_normal['filtered'][out_normal['rpeaks'][:10]], 'ro', label='R-peaks')
axes[0].set_title('Filtered ECG Data with R-peaks - Normal Condition')
axes[0].set_xlabel('Time (s)')
axes[0].set_ylabel('Amplitude')
axes[0].legend()
axes[0].grid(True)

# Plotting the filtered ECG data with detected R-peaks for atrial fibrillation (Afib) condition
axes[1].plot(out_af['ts'][:8000], out_af['filtered'][:8000], label='Filtered Afib ECG', color='red')
axes[1].plot(out_af['ts'][out_af['rpeaks'][:10]], out_af['filtered'][out_af['rpeaks'][:10]], 'ro', label='R-peaks')
axes[1].set_title('Filtered ECG Data with R-peaks - Atrial Fibrillation (Afib) Condition')
axes[1].set_xlabel('Time (s)')
axes[1].set_ylabel('Amplitude')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()


We will use these R-peaks later to calculate features which will help us distinguish between healthy and Afib functions. 

## <span style="color:#00aba1;"> 2.3. Extracting Heartbeat Templates </span>


Another thing we can examine using the `biosppy` ECG function are heartbeat templates. 

Heartbeat templates provide a standardized representation of individual heartbeats in the ECG signal. 

Visualizing these templates can help us understand the structure and characteristics of each heartbeat, as well as variations between heartbeats, which can indicate physiological and electrical characteristics of the heart.

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(15, 12))

# Plotting the extracted heartbeat templates for normal condition
for template in out_normal['templates'][:5]:
    axes[0].plot(out_normal['templates_ts'], template, label='Heartbeat Template - Normal', color='blue', alpha=0.7)
axes[0].set_title('Extracted Heartbeat Templates - Normal Condition')
axes[0].set_xlabel('Time (s)')
axes[0].set_ylabel('Amplitude')
axes[0].grid(True)

# Plotting the extracted heartbeat templates for atrial fibrillation (AF) condition
for template in out_af['templates'][:5]:
    axes[1].plot(out_af['templates_ts'], template, label='Heartbeat Template - AF', color='red', alpha=0.7)
axes[1].set_title('Extracted Heartbeat Templates - Atrial Fibrillation (AF) Condition')
axes[1].set_xlabel('Time (s)')
axes[1].set_ylabel('Amplitude')
axes[1].grid(True)

plt.tight_layout()
plt.show()


## <span style="color:#00aba1;"> 2.3. Extracting R-R Intervals & Heart Rate </span>

The R-R intervals (RRI) correspond to the time difference between consecutive R-peaks of the ECG.

The heart rate (HR), often expressed in beats per minute (bpm), represents the frequency of the cardiac cycle.

RRI and HR provide insights into the heart's activity and can be used to detect abnormalities or conditions like tachycardia (increased heart rate) or bradycardia (decreased heart rate).

Let's visualize the RRI and hr over time for both the normal and atrial fibrillation (AF) conditions.

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(15, 12))

# Plotting instantaneous heart rate for both conditions
axes[0].plot(out_normal['heart_rate_ts'], out_normal['heart_rate'], 'b-', label='Heart Rate - Normal', linewidth=2)
axes[0].plot(out_af['heart_rate_ts'], out_af['heart_rate'], 'r-', label='Heart Rate - Afib', linewidth=2)
axes[0].set_title('Instantaneous Heart Rate - Normal vs. Atrial Fibrillation (Afib)')
axes[0].set_xlabel('Time (s)')
axes[0].set_ylabel('Heart Rate (bpm)')
axes[0].grid(True)
axes[0].legend()

# Compute R-R intervals for both conditions
rr_intervals_normal = np.diff(out_normal['rpeaks']) / fs
rr_intervals_af = np.diff(out_af['rpeaks']) / fs

# Time stamps for R-R intervals both conditions
rr_times_normal = out_normal['rpeaks'][1:] / fs
rr_times_af = out_af['rpeaks'][1:] / fs

# Plotting R-R intervals for both conditions
axes[1].plot(rr_times_normal, rr_intervals_normal, 'b-', label='R-R Intervals - Normal', linewidth=2)
axes[1].plot(rr_times_af, rr_intervals_af, 'r-', label='R-R Intervals - Afib', linewidth=2)
axes[1].set_title('R-R Intervals - Normal vs. Atrial Fibrillation (Afib)')
axes[1].set_xlabel('Time (s)')
axes[1].set_ylabel('R-R Interval (s)')
axes[1].grid(True)
axes[1].legend()

plt.tight_layout()
plt.show()


We can also represent the R-R intervals with histograms:

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(15, 12))

# Plotting histogram for R-R intervals of the normal condition
axes[0].hist(rr_intervals_normal, bins=50, color='blue', edgecolor='black', alpha=0.7)
axes[0].set_title('Histogram of R-R Intervals - Normal Condition')
axes[0].set_xlabel('R-R Interval (s)')
axes[0].set_ylabel('Frequency')
axes[0].grid(True)

# Plotting histogram for R-R intervals of the atrial fibrillation (Afib) condition
axes[1].hist(rr_intervals_af, bins=50, color='red', edgecolor='black', alpha=0.7)
axes[1].set_title('Histogram of R-R Intervals - Atrial Fibrillation (Afib) Condition')
axes[1].set_xlabel('R-R Interval (s)')
axes[1].set_ylabel('Frequency')
axes[1].grid(True)

plt.tight_layout()
plt.show()

> ✏️ *QUESTIONS:*
> -  Analyze the signals above. How do the ECG signals, heartbeat templates, RRI and HR differ between the healty and atrial fibrillation data, and how can we account for these observed differences from a pathological perspective?
> -  Based only on your analysis of the data we have seen so far, what characteristics or features could useful to discern between these conditions? 
 

# <span style="color:#00aba1;"> 3. Heart Rate Variability (HRV) </span>

Heart Rate Variability (HRV) is a measure of the variation in time between successive heartbeats. 

HRV is influenced by the autonomic nervous system, which includes sympathetic and parasympathetic nerve branches. These branches innervate parts of the heart's atria. The sympathetic branch, when activated, accelerates the heart rate, whereas the parasympathetic branch slows it down.

Apart from the natural heart rate fluctuations associated with breathing, HRV can also to be affected by other factors, such as mental stress, sleep patterns, fatigue, chronic illnesses, and more. Curiously, elevated HRV is sometimes associated with better cardiovascular health, since it indicates the physiological system is more resilient, flexible, and efficient in how it regulates heart rate.


HRV can be analyzed in both the time domain and the frequency domain, each providing different insights into heart function. We will describe some of them below.

## <span style="color:#00aba1;"> 3.1. Time-Domain HRV Features </span>

Having previously calculated the R-R intervals, we can now compute statistical metrics to reflect the variation in these intervals over time. The most common time-domain HRV features are:

- **Mean RR Interval**: Average of all RR intervals.
- **SDNN (Standard Deviation of NN intervals)**: Measures the variability of all RR intervals.
- **RMSSD (Root Mean Square of Successive Differences)**: Measures the short-term variability of RR intervals.

In [ ]:
def time_domain_hrv_features(rr_intervals):

    # Time-domain HRV features
    mean_rr = np.mean(rr_intervals)
    sdnn = np.std(rr_intervals)
    rmssd = np.sqrt(np.mean(np.square(np.diff(rr_intervals))))

    return mean_rr, sdnn, rmssd

mean_rr_normal, sdnn_normal, rmssd_normal = time_domain_hrv_features(rr_intervals_normal)
mean_rr_af, sdnn_af, rmssd_af = time_domain_hrv_features(rr_intervals_af)
    
print('Time-Domain HRV Features - Normal')
print(f'Mean = {mean_rr_normal}, SD = {sdnn_normal}, RMSSD = {rmssd_normal}')
print('')
print('Time-Domain HRV Features - Atrial Fibrillation')
print(f'Mean = {mean_rr_af}, SD = {sdnn_af}, RMSSD = {rmssd_af}')

## <span style="color:#00aba1;"> 3.2. Frequency-Domain HRV Features </span>

We can also compute statistical metrics to reflect frequency information. The most common frequency-domain HRV features are:

- **LF (Low Frequency)**: Power in the low-frequency range (0.04-0.15 Hz). It is associated with both sympathetic and parasympathetic activity.
- **HF (High Frequency)**: Power in the high-frequency range (0.15-0.4 Hz). It is mainly associated with parasympathetic activity.
- **LF/HF Ratio**: Ratio of LF to HF power. It provides information about the balance between sympathetic and parasympathetic activity.

In [ ]:
from astropy.timeseries import LombScargle

def frequency_domain_hrv_features(rr_intervals, fs=1000.0):

    # Frequency-domain HRV features
    time_rri = np.cumsum(rr_intervals)
    frequency, power = LombScargle(time_rri, rr_intervals).autopower(minimum_frequency=0.01, maximum_frequency=0.5*2*np.pi)
    frequency = frequency / (2 * np.pi)

    lf = np.trapz(power[np.logical_and(frequency >= 0.04, frequency <= 0.15)])
    hf = np.trapz(power[np.logical_and(frequency >= 0.15, frequency <= 0.4)])
    lf_hf_ratio = lf / hf if hf != 0 else np.nan  # return np.nan if hf is zero

    return lf, hf, lf_hf_ratio

lf_normal, hf_normal, lf_hf_ratio_normal = frequency_domain_hrv_features(rr_intervals_normal)
lf_af, hf_af, lf_hf_ratio_af = frequency_domain_hrv_features(rr_intervals_af)
    
print('Frequency-Domain HRV Features - Normal')
print(f'LF = {lf_normal}, HF = {hf_normal}, LF/HF Ratio = {lf_hf_ratio_normal}')
print('')
print('Frequency-Domain HRV Features - Atrial Fibrillation')
print(f'LF = {lf_af}, HF = {hf_af},  LF/HF Ratio = {lf_hf_ratio_af}')


## <span style="color:#00aba1;"> 3.3. Poincaré Plot </span>

A Poincaré plot is a graphical representation of the correlation between successive RR intervals that can provide additional insights into the patterns and structures within the RR intervals. 

From this plot we can extract features, such as the standard descriptor 1 (SD1), which describes the short-term variability in the R-R intervals, and the standard descriptor 2 (SD2), which describes the long-term variability. SD1 mostly represents parasympathetic heart influence, whereas SD2 reflects sympathetic and parasympathetic inputs to the heart.

![poincare](https://raw.githubusercontent.com/scientisst/notebooks/master/F.Machine%20Learning%20and%20Knowledge%20Extraction/F005%20Biosignal%20Feature%20Extraction/_Resources/poincare.png)


In [ ]:
def plot_poincare(out):
    print('Plot Poincaré')
        
    fs=1000
    
    rpeaks = out['rpeaks']/fs*1000 # convert sample indexes to ms
    rri = np.diff(rpeaks)
    
    x = rri[:-1]
    y = rri[1:]
    rr_mean = np.mean(rri)
    
    # Plot
    fig, ax = plt.subplots(figsize=(6,6))
    ax.set_title('Poincaré Plot', fontsize=14)
    ax.set_xlabel('$RR_i$ (ms)', fontsize=12)
    ax.set_ylabel('$RR_{i+1}$ (ms)', fontsize=12)
    
    # Plot Poincaré data points
    ax.scatter(x, y, marker='.', color='blue', alpha=0.5,s=100)
    ax.set_xlim([np.min(rri) - 50, np.max(rri) + 50])
    ax.set_ylim([np.min(rri) - 50, np.max(rri) + 50])
    ax.set_aspect(1./ax.get_data_ratio())
    
    
    # Draw RRi+1=RRi
    lims = [np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
        np.max([ax.get_xlim(), ax.get_ylim()])]  # max of both axes
    
    ax.plot(lims, lims, color='grey', linestyle='--')
    
    
    # Draw ellipse
    sd1 = np.std(np.subtract(x, y) / np.sqrt(2))
    sd2 = np.std(np.add(x, y) / np.sqrt(2))
    area = np.pi * sd1 * sd2
    
    
    ellipse_ = matplotlib.patches.Ellipse((rr_mean, rr_mean), sd1 * 2, sd2 * 2, angle=-45, edgecolor='black', facecolor="None")
    ax.add_artist(ellipse_)
    
    ax.grid()
    
    print(f'SD1 = {sd1} \nSD2 = {sd2} \nArea = {area} \nSD1/SD2 = {sd1/sd2}')
    
# Plot Poincaré plot for normal condition
plot_poincare(out_normal)

# Plot Poincaré plot for AF condition
plot_poincare(out_af)

# <span style="color:#00aba1;"> 3. Visualizing Features </span>

Now, in order to visualize these features with more datapoints, we will be using ECG recordings of a real dataset acquired using AliveCor's KardiaMobile, available in <a href="https://physionet.org/content/challenge-2017/1.0.0/">PhysioNet.org</a>, which contain 8528 single short ECG lead recordings sampled at 300 Hz. These are:

* *ecg_nsr* containing 100 10-second ECG recordings in Normal Sinus Rhythm.
* *ecg_af* containing 100 10-second ECG recordings in Atrial Fibrillation.

The recordings were extracted from the database and are already preprocessed using BioSPPy.

In [ ]:
# Load the ECG data
fs = 300. # Hz
ecg_nsr = np.loadtxt('https://raw.githubusercontent.com/scientisst/notebooks/master/F.Machine%20Learning%20and%20Knowledge%20Extraction/F006%20Introduction%20to%20Classification/_Resources/NSR_arrays.csv')
ecg_af = np.loadtxt('https://raw.githubusercontent.com/scientisst/notebooks/master/F.Machine%20Learning%20and%20Knowledge%20Extraction/F006%20Introduction%20to%20Classification/_Resources/AFib_arrays.csv')


Now let's extract HRV features for the new data:

In [ ]:
from astropy.timeseries import LombScargle
import pandas as pd


def compute_hrv_features(ecg, fs):
    out = bp.signals.ecg.ecg(ecg, fs, show=False)
    rpeaks = out['rpeaks']

    # Compute RR intervals
    rr_intervals = np.diff(rpeaks) / fs

    # Time-domain HRV features
    mean_rr = np.mean(rr_intervals)
    sdnn = np.std(rr_intervals)
    rmssd = np.sqrt(np.mean(np.square(np.diff(rr_intervals))))

    # Frequency-domain HRV features
    time_rri = np.cumsum(rr_intervals)
    frequency, power = LombScargle(time_rri, rr_intervals).autopower(minimum_frequency=0.01, maximum_frequency=0.5*2*np.pi)
    frequency = frequency / (2 * np.pi)

    lf = np.trapz(power[np.logical_and(frequency >= 0.04, frequency <= 0.15)])
    hf = np.trapz(power[np.logical_and(frequency >= 0.15, frequency <= 0.4)])
    lf_hf_ratio = lf / hf if hf != 0 else np.nan  # return np.nan if hf is zero

    
    rpeaks_ms = out['rpeaks']/fs*1000  # convert sample indexes to ms
    rri = np.diff(rpeaks)
    
    x = rri[:-1]
    y = rri[1:]
    
    sd1 = np.std(np.subtract(x, y) / np.sqrt(2))
    sd2 = np.std(np.add(x, y) / np.sqrt(2))
    area = np.pi * sd1 * sd2
    
    sd1_sd2_ratio = sd1/sd2


    return {
        "mean_rr": mean_rr,
        "sdnn": sdnn,
        "rmssd": rmssd,
        "lf": lf,
        "hf": hf,
        "lf_hf_ratio": lf_hf_ratio,
        "sd1": sd1,
        "sd2": sd2,
        "area": area,
        "sd1_sd2_ratio": sd1_sd2_ratio
    }

# Compute hrv metrics
NSR_hrv = [compute_hrv_features(i, fs) for i in ecg_nsr]
AFib_hrv = [compute_hrv_features(i, fs) for i in ecg_af]

# Combine NSR and AFib features
X = NSR_hrv + AFib_hrv
y = [0] * len(NSR_hrv) + [1] * len(AFib_hrv)  # 0 for NSR, 1 for AFib

# Convert training data to DataFrame for visualization
feature_vector = pd.DataFrame(X, columns=["mean_rr", "sdnn", "rmssd", "lf", "hf", "lf_hf_ratio", "sd1", "sd2", "area", "sd1_sd2_ratio"])

feature_vector

One of the most powerful tools for visualizing high-dimensional datasets with multiple features is the Seaborn's pairplot. It displays pairwise relationships between features. This way, we can immediately see the distributions of single variables and relationships between every two features.

In [ ]:
import seaborn as sns
import pandas as pd

# Set the seaborn context to adjust font scales and other aesthetics
sns.set_context("talk")

# Map numeric labels to their corresponding string names
label_mapping = {0: "NSR", 1: "AFib"}
feature_vector['label'] = pd.Series(y).map(label_mapping)

# Use plot_kws to adjust dot size, and corner to plot only the lower half
g = sns.pairplot(feature_vector, hue='label', plot_kws={'s': 20}, corner=True)

# Enhance the font size of x and y labels
for ax in g.axes.flat: 
    if ax:
        ax.set_xlabel(ax.get_xlabel(), fontsize='large')
        ax.set_ylabel(ax.get_ylabel(), fontsize='large')

# Show the plot
plt.show()

> ✏️ *QUESTIONS:*
> -  Analyze the distribution curves for each individual feature and scatter graphs for pairs of features. Which individual HRV features and which pairs of HRV features seem to be the best at distinguishing between NSR and Afib, and which seem to be the worse? How can we account for these observed differences from a pathological perspective?
    
> -  In this plot we only see pairs of features. However, classifiers often use multiple features. Why?
     

# <span style="color:#00aba1;"> Further Reading </span>

* Dong, J. (2016). *The role of heart rate variability in sports physiology (Review)*. Experimental and Therapeutic Medicine, 11, 1531-1536. https://doi.org/10.3892/etm.2016.3104
* Makivic , B., &amp; Bauer, P. *Heart Rate Variability Analysis in Sport*. Aspetar Sports Medicine Journal - Heart rate variability analysis in sport. Retrieved June 6, 2022, from https://www.aspetar.com/journal/viewarticle.aspx?id=248#.Yp4f2qjMJPZ 
* Valencell. *Heart Rate Variability Review*. Retrieved June 6, 2022, from https://valencell.com/blog/heart-rate-variability-review/

<div style="width: 100%; ">
<div style="background:#00a0e4;color:white;font-family:'arial', monospace; text-align: center; padding: 50px 0; border-radius:10px; height:10px; width:100%; float:left " >
<span style="font-size:17px;position:relative; top:-20px"> Hope you enjoy this lab! </span> 
<br>
<span style="font-size:12px;position:relative; top:-25px"> For any questions, email me at: vicente.garcao@tecnico.ulisboa.pt <span style="font-size:14px;position:relative;COLOR:WHITE"> </span></span> 
</div>
    
    

![scientisst-notebooks_bottom-banner](https://raw.githubusercontent.com/scientisst/notebooks/master/_Resources/bottom-banner.png)